In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
import pandas as pd
from scipy.stats import skew 
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline
train_data = pd.read_csv("Data/CE802_P3_Data.csv",sep=',')
test_data = pd.read_csv("Data/CE802_P3_Test.csv",sep=',')
file_name="Data/CE802_P3_Test_LR.csv"


Bad key "text.kerning_factor" on line 4 in
C:\Conda\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
train_data

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,Target
0,16.56,12.42,-236.06,Rest,-98.88,529.56,4.54,379.54,1,1,7.30,High,-15085.87,-12.93,-39.42,1734.58,3616.82
1,11.72,12.46,-190.06,Rest,-59.22,493.11,0.05,402.78,5,3,-1.28,Very low,-15782.44,-8.55,-35.61,1672.70,3342.88
2,4.34,2.74,-201.20,UK,-228.48,563.79,1.22,147.35,4,4,8.28,Low,-10526.01,-9.66,-29.10,1462.86,0.00
3,12.76,2.58,-282.26,UK,-173.28,536.94,0.25,113.49,4,3,6.26,Low,-8327.14,-19.23,-34.59,809.46,1742.65
4,11.10,9.82,-242.86,USA,-193.14,617.52,9.15,343.64,8,6,-6.88,Very low,-14434.13,-9.45,-46.14,1435.90,373.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,5.20,4.70,-218.72,Rest,-224.04,539.91,8.23,-232.93,2,3,12.32,Very high,-16977.67,5.01,-29.67,1420.16,1515.56
1496,12.10,2.98,-144.42,UK,-76.05,736.65,0.48,90.41,3,6,7.40,Medium,-13927.89,0.66,-36.06,1321.82,1528.48
1497,4.12,4.36,-160.30,Europe,-230.43,439.86,0.69,-224.44,3,3,7.02,Medium,-18373.26,-7.68,-39.42,1905.86,323.67
1498,1.58,13.50,-159.60,USA,-233.85,572.58,0.29,-126.79,1,5,1.68,Very high,-22863.72,-3.12,-54.36,2424.42,109.19


In [3]:
# def safe_log10(x, eps=1e-10):    
#     result = np.where(x > eps, x, -1).astype(np.float)     
#     np.log10(result, out=result, where=result > 0)     
#     return result

In [4]:
# y = safe_log10(train_data['Target']).reshape(-1,1) # Apply natural log function to the target


In [5]:
# y

In [6]:
test_data

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,Target
0,28.70,0.82,-115.06,USA,-280.95,581.10,0.06,214.35,2,2,-0.88,Low,-14794.01,-15.09,-35.61,1578.68,NaN
1,5.40,6.90,-170.24,Rest,-228.00,330.54,0.31,-164.98,1,9,0.28,Medium,-11632.88,0.72,-29.73,1647.02,NaN
2,21.96,4.40,-219.12,Rest,-59.67,662.28,4.98,281.54,6,5,5.82,Low,-12631.29,-23.61,-32.73,1449.14,NaN
3,10.52,1.02,-184.14,UK,-211.23,506.55,0.57,-311.96,7,5,1.92,High,-26664.88,1.80,-22.92,911.98,NaN
4,2.00,0.58,-190.56,Rest,-284.82,649.41,0.05,291.47,8,3,-0.02,Low,-9901.41,-13.47,-48.81,2251.04,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,18.42,8.22,-213.52,Rest,-189.33,569.43,0.09,-315.11,2,3,8.64,Very high,-16759.93,-13.56,-37.08,869.80,NaN
1496,12.06,14.46,-251.02,Rest,-87.90,640.32,3.67,201.37,7,8,9.34,Very low,-11645.46,-4.35,-56.10,2040.40,NaN
1497,6.54,10.42,-156.28,Europe,-221.82,462.30,0.00,-267.50,2,6,5.00,Low,-23752.87,-8.37,-21.27,1130.64,NaN
1498,8.06,13.34,-211.82,UK,-483.24,766.38,0.19,140.38,4,6,5.34,Medium,-13748.25,-9.21,-18.96,1261.38,NaN


In [7]:
cleanup = {"F12": {"Very low":1, "Low":2, "Medium":3, "High":4, "Very high":5}}
train_data = train_data.replace(cleanup)
train_data

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,Target
0,16.56,12.42,-236.06,Rest,-98.88,529.56,4.54,379.54,1,1,7.30,4,-15085.87,-12.93,-39.42,1734.58,3616.82
1,11.72,12.46,-190.06,Rest,-59.22,493.11,0.05,402.78,5,3,-1.28,1,-15782.44,-8.55,-35.61,1672.70,3342.88
2,4.34,2.74,-201.20,UK,-228.48,563.79,1.22,147.35,4,4,8.28,2,-10526.01,-9.66,-29.10,1462.86,0.00
3,12.76,2.58,-282.26,UK,-173.28,536.94,0.25,113.49,4,3,6.26,2,-8327.14,-19.23,-34.59,809.46,1742.65
4,11.10,9.82,-242.86,USA,-193.14,617.52,9.15,343.64,8,6,-6.88,1,-14434.13,-9.45,-46.14,1435.90,373.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,5.20,4.70,-218.72,Rest,-224.04,539.91,8.23,-232.93,2,3,12.32,5,-16977.67,5.01,-29.67,1420.16,1515.56
1496,12.10,2.98,-144.42,UK,-76.05,736.65,0.48,90.41,3,6,7.40,3,-13927.89,0.66,-36.06,1321.82,1528.48
1497,4.12,4.36,-160.30,Europe,-230.43,439.86,0.69,-224.44,3,3,7.02,3,-18373.26,-7.68,-39.42,1905.86,323.67
1498,1.58,13.50,-159.60,USA,-233.85,572.58,0.29,-126.79,1,5,1.68,5,-22863.72,-3.12,-54.36,2424.42,109.19


In [8]:
test = test_data.replace(cleanup)
test

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,Target
0,28.70,0.82,-115.06,USA,-280.95,581.10,0.06,214.35,2,2,-0.88,2,-14794.01,-15.09,-35.61,1578.68,NaN
1,5.40,6.90,-170.24,Rest,-228.00,330.54,0.31,-164.98,1,9,0.28,3,-11632.88,0.72,-29.73,1647.02,NaN
2,21.96,4.40,-219.12,Rest,-59.67,662.28,4.98,281.54,6,5,5.82,2,-12631.29,-23.61,-32.73,1449.14,NaN
3,10.52,1.02,-184.14,UK,-211.23,506.55,0.57,-311.96,7,5,1.92,4,-26664.88,1.80,-22.92,911.98,NaN
4,2.00,0.58,-190.56,Rest,-284.82,649.41,0.05,291.47,8,3,-0.02,2,-9901.41,-13.47,-48.81,2251.04,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,18.42,8.22,-213.52,Rest,-189.33,569.43,0.09,-315.11,2,3,8.64,5,-16759.93,-13.56,-37.08,869.80,NaN
1496,12.06,14.46,-251.02,Rest,-87.90,640.32,3.67,201.37,7,8,9.34,1,-11645.46,-4.35,-56.10,2040.40,NaN
1497,6.54,10.42,-156.28,Europe,-221.82,462.30,0.00,-267.50,2,6,5.00,2,-23752.87,-8.37,-21.27,1130.64,NaN
1498,8.06,13.34,-211.82,UK,-483.24,766.38,0.19,140.38,4,6,5.34,3,-13748.25,-9.21,-18.96,1261.38,NaN


In [9]:
train_data = pd.get_dummies(train_data, columns= ['F4'])
train_data.head()

,F1,F2,F3,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,Target,F4_Europe,F4_Rest,F4_UK,F4_USA
0,16.56,12.42,-236.06,-98.88,529.56,4.54,379.54,1,1,7.30,4,-15085.87,-12.93,-39.42,1734.58,3616.82,0,1,0,0
1,11.72,12.46,-190.06,-59.22,493.11,0.05,402.78,5,3,-1.28,1,-15782.44,-8.55,-35.61,1672.70,3342.88,0,1,0,0
2,4.34,2.74,-201.20,-228.48,563.79,1.22,147.35,4,4,8.28,2,-10526.01,-9.66,-29.10,1462.86,0.00,0,0,1,0
3,12.76,2.58,-282.26,-173.28,536.94,0.25,113.49,4,3,6.26,2,-8327.14,-19.23,-34.59,809.46,1742.65,0,0,1,0
4,11.10,9.82,-242.86,-193.14,617.52,9.15,343.64,8,6,-6.88,1,-14434.13,-9.45,-46.14,1435.90,373.56,0,0,0,1


In [10]:
test = pd.get_dummies(test, columns= ['F4'])
test.head()

,F1,F2,F3,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,Target,F4_Europe,F4_Rest,F4_UK,F4_USA
0,28.70,0.82,-115.06,-280.95,581.10,0.06,214.35,2,2,-0.88,2,-14794.01,-15.09,-35.61,1578.68,NaN,0,0,0,1
1,5.40,6.90,-170.24,-228.00,330.54,0.31,-164.98,1,9,0.28,3,-11632.88,0.72,-29.73,1647.02,NaN,0,1,0,0
2,21.96,4.40,-219.12,-59.67,662.28,4.98,281.54,6,5,5.82,2,-12631.29,-23.61,-32.73,1449.14,NaN,0,1,0,0
3,10.52,1.02,-184.14,-211.23,506.55,0.57,-311.96,7,5,1.92,4,-26664.88,1.80,-22.92,911.98,NaN,0,0,1,0
4,2.00,0.58,-190.56,-284.82,649.41,0.05,291.47,8,3,-0.02,2,-9901.41,-13.47,-48.81,2251.04,NaN,0,1,0,0


In [11]:
# for i in range(1,train_data.shape[1]):
#     col='F'+str(i)
#     print("Col #",col)
#     plt.scatter(train_data[col], train_data['Target'])
#     plt.show()
#     kwargs = dict(alpha=0.5, bins=100)
#     plt.hist(train_data[col], **kwargs, color='g', label='Ideal')
#     plt.show()

In [12]:
train_data

,F1,F2,F3,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,Target,F4_Europe,F4_Rest,F4_UK,F4_USA
0,16.56,12.42,-236.06,-98.88,529.56,4.54,379.54,1,1,7.30,4,-15085.87,-12.93,-39.42,1734.58,3616.82,0,1,0,0
1,11.72,12.46,-190.06,-59.22,493.11,0.05,402.78,5,3,-1.28,1,-15782.44,-8.55,-35.61,1672.70,3342.88,0,1,0,0
2,4.34,2.74,-201.20,-228.48,563.79,1.22,147.35,4,4,8.28,2,-10526.01,-9.66,-29.10,1462.86,0.00,0,0,1,0
3,12.76,2.58,-282.26,-173.28,536.94,0.25,113.49,4,3,6.26,2,-8327.14,-19.23,-34.59,809.46,1742.65,0,0,1,0
4,11.10,9.82,-242.86,-193.14,617.52,9.15,343.64,8,6,-6.88,1,-14434.13,-9.45,-46.14,1435.90,373.56,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,5.20,4.70,-218.72,-224.04,539.91,8.23,-232.93,2,3,12.32,5,-16977.67,5.01,-29.67,1420.16,1515.56,0,1,0,0
1496,12.10,2.98,-144.42,-76.05,736.65,0.48,90.41,3,6,7.40,3,-13927.89,0.66,-36.06,1321.82,1528.48,0,0,1,0
1497,4.12,4.36,-160.30,-230.43,439.86,0.69,-224.44,3,3,7.02,3,-18373.26,-7.68,-39.42,1905.86,323.67,1,0,0,0
1498,1.58,13.50,-159.60,-233.85,572.58,0.29,-126.79,1,5,1.68,5,-22863.72,-3.12,-54.36,2424.42,109.19,0,0,0,1


In [13]:
test

,F1,F2,F3,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,Target,F4_Europe,F4_Rest,F4_UK,F4_USA
0,28.70,0.82,-115.06,-280.95,581.10,0.06,214.35,2,2,-0.88,2,-14794.01,-15.09,-35.61,1578.68,NaN,0,0,0,1
1,5.40,6.90,-170.24,-228.00,330.54,0.31,-164.98,1,9,0.28,3,-11632.88,0.72,-29.73,1647.02,NaN,0,1,0,0
2,21.96,4.40,-219.12,-59.67,662.28,4.98,281.54,6,5,5.82,2,-12631.29,-23.61,-32.73,1449.14,NaN,0,1,0,0
3,10.52,1.02,-184.14,-211.23,506.55,0.57,-311.96,7,5,1.92,4,-26664.88,1.80,-22.92,911.98,NaN,0,0,1,0
4,2.00,0.58,-190.56,-284.82,649.41,0.05,291.47,8,3,-0.02,2,-9901.41,-13.47,-48.81,2251.04,NaN,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,18.42,8.22,-213.52,-189.33,569.43,0.09,-315.11,2,3,8.64,5,-16759.93,-13.56,-37.08,869.80,NaN,0,1,0,0
1496,12.06,14.46,-251.02,-87.90,640.32,3.67,201.37,7,8,9.34,1,-11645.46,-4.35,-56.10,2040.40,NaN,0,1,0,0
1497,6.54,10.42,-156.28,-221.82,462.30,0.00,-267.50,2,6,5.00,2,-23752.87,-8.37,-21.27,1130.64,NaN,1,0,0,0
1498,8.06,13.34,-211.82,-483.24,766.38,0.19,140.38,4,6,5.34,3,-13748.25,-9.21,-18.96,1261.38,NaN,0,0,1,0


In [14]:
X = train_data.drop('Target', axis=1)
y = train_data['Target']

X_test=test.drop('Target',axis=1)

In [15]:
X

,F1,F2,F3,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F4_Europe,F4_Rest,F4_UK,F4_USA
0,16.56,12.42,-236.06,-98.88,529.56,4.54,379.54,1,1,7.30,4,-15085.87,-12.93,-39.42,1734.58,0,1,0,0
1,11.72,12.46,-190.06,-59.22,493.11,0.05,402.78,5,3,-1.28,1,-15782.44,-8.55,-35.61,1672.70,0,1,0,0
2,4.34,2.74,-201.20,-228.48,563.79,1.22,147.35,4,4,8.28,2,-10526.01,-9.66,-29.10,1462.86,0,0,1,0
3,12.76,2.58,-282.26,-173.28,536.94,0.25,113.49,4,3,6.26,2,-8327.14,-19.23,-34.59,809.46,0,0,1,0
4,11.10,9.82,-242.86,-193.14,617.52,9.15,343.64,8,6,-6.88,1,-14434.13,-9.45,-46.14,1435.90,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,5.20,4.70,-218.72,-224.04,539.91,8.23,-232.93,2,3,12.32,5,-16977.67,5.01,-29.67,1420.16,0,1,0,0
1496,12.10,2.98,-144.42,-76.05,736.65,0.48,90.41,3,6,7.40,3,-13927.89,0.66,-36.06,1321.82,0,0,1,0
1497,4.12,4.36,-160.30,-230.43,439.86,0.69,-224.44,3,3,7.02,3,-18373.26,-7.68,-39.42,1905.86,1,0,0,0
1498,1.58,13.50,-159.60,-233.85,572.58,0.29,-126.79,1,5,1.68,5,-22863.72,-3.12,-54.36,2424.42,0,0,0,1


In [16]:
cols=[col for col in X.columns]
cols

['F1',
 'F2',
 'F3',
 'F5',
 'F6',
 'F7',
 'F8',
 'F9',
 'F10',
 'F11',
 'F12',
 'F13',
 'F14',
 'F15',
 'F16',
 'F4_Europe',
 'F4_Rest',
 'F4_UK',
 'F4_USA']

In [17]:
import itertools
xs = [cols[i:j] for i, j in itertools.combinations(range(len(cols)+1), 2)]

In [18]:
# X[['F1', 'F2', 'F3', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F4_Europe', 'F4_Rest', 'F4_UK']]

In [19]:
max_score=-10000000000
best_comb=None
scaler = StandardScaler()
y=scaler.fit_transform(y.values.reshape(-1,1))
for comb in xs:
    print(comb)
    selection = X[comb]
    selection=scaler.fit_transform(selection)


    from sklearn.model_selection import train_test_split
    X_train, X_valid, y_train, y_valid =train_test_split(selection, y,  random_state=1, test_size = 0.2)
#     pt = PowerTransformer(method='yeo-johnson')
#     pt.fit(X_train)                    ## Fit the PT on training data
#     X_train = pt.transform(X_train)    ## Then apply on all data
#     X_valid = pt.transform(X_valid)
#     pt.fit(test)
#     test = pt.transform(test)
    lr = LinearRegression()
    lr.fit(X_train,y_train)      
    cv_score = cross_val_score(lr, X_train, y_train, cv = 5,scoring='r2')
    print("CV mean score: ", cv_score.mean())
    if cv_score.mean()>max_score:
        max_score=cv_score.mean()
        best_comb=comb
print("Best score:",best_comb,max_score)

['F1']
CV mean score:  0.11053115158860201
['F1', 'F2']
CV mean score:  0.24268771059375854
['F1', 'F2', 'F3']
CV mean score:  0.24200718227311774
['F1', 'F2', 'F3', 'F5']
CV mean score:  0.40564649255980906
['F1', 'F2', 'F3', 'F5', 'F6']
CV mean score:  0.40561009972383466
['F1', 'F2', 'F3', 'F5', 'F6', 'F7']
CV mean score:  0.3987777964780636
['F1', 'F2', 'F3', 'F5', 'F6', 'F7', 'F8']
CV mean score:  0.48811295887371414
['F1', 'F2', 'F3', 'F5', 'F6', 'F7', 'F8', 'F9']
CV mean score:  0.5265955419689206
['F1', 'F2', 'F3', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10']
CV mean score:  0.5815852098278849
['F1', 'F2', 'F3', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11']
CV mean score:  0.5804553598872946
['F1', 'F2', 'F3', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12']
CV mean score:  0.621022859816047
['F1', 'F2', 'F3', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'F13']
CV mean score:  0.6195618265524799
['F1', 'F2', 'F3', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'F13', 'F14']
C

CV mean score:  0.2083187112562761
['F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16']
CV mean score:  0.28470054888003116
['F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F4_Europe']
CV mean score:  0.2846199602524263
['F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F4_Europe', 'F4_Rest']
CV mean score:  0.34360301044215236
['F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F4_Europe', 'F4_Rest', 'F4_UK']
CV mean score:  0.362029173157593
['F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F4_Europe', 'F4_Rest', 'F4_UK', 'F4_USA']
CV mean score:  0.36194875702603274
['F8']
CV mean score:  0.08201624057099499
['F8', 'F9']
CV mean score:  0.11486036559845651
['F8', 'F9', 'F10']
CV mean score:  0.17214840748755283
['F8', 'F9', 'F10', 'F11']
CV mean score:  0.17205431303965582
['F8', 'F9', 'F10', 'F11', 'F12']
CV mean score:  0.2223175756845222
['F8', 'F9', 'F10', 'F11', 'F12', 'F13']
CV mean score:

In [20]:
y_predict = lr.predict(X_valid)   
mae = sum((y_predict - y_valid)**2) /len(y_predict)

print(mae)

[0.88480456]


In [21]:
y_predict

array([[ 0.14534925],
       [ 0.14534925],
       [ 0.14534925],
       [ 0.14534925],
       [ 0.14534925],
       [ 0.14534925],
       [-0.4108267 ],
       [-0.4108267 ],
       [ 0.14534925],
       [ 0.14534925],
       [ 0.14534925],
       [ 0.14534925],
       [ 0.14534925],
       [ 0.14534925],
       [ 0.14534925],
       [ 0.14534925],
       [ 0.14534925],
       [ 0.14534925],
       [ 0.14534925],
       [-0.4108267 ],
       [ 0.14534925],
       [ 0.14534925],
       [ 0.14534925],
       [ 0.14534925],
       [ 0.14534925],
       [-0.4108267 ],
       [-0.4108267 ],
       [ 0.14534925],
       [ 0.14534925],
       [ 0.14534925],
       [ 0.14534925],
       [ 0.14534925],
       [ 0.14534925],
       [ 0.14534925],
       [-0.4108267 ],
       [ 0.14534925],
       [ 0.14534925],
       [ 0.14534925],
       [ 0.14534925],
       [ 0.14534925],
       [ 0.14534925],
       [ 0.14534925],
       [-0.4108267 ],
       [ 0.14534925],
       [ 0.14534925],
       [-0

In [22]:
# Create a dataframe that output the Date, the Actual and the predicted values
df = pd.DataFrame({'Record':range(0,len(y_valid)),'Actual': y_valid, 'Predicted': y_predict})
df1 = df.head(50)
 
df1.set_index('Record',inplace=True)
df1

Exception: Data must be 1-dimensional

In [ ]:
error = df1['Actual'] - df1['Predicted']

# Plot the error term between the actual and predicted values for the last 25 days
 
error.plot(kind='bar',figsize=(15,10))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.xticks(rotation=45)
plt.show()

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_valid,y_predict)

In [ ]:
y_predict = lr.predict(X_test)   
y_predict
for i in range(0,len(y_predict)):
    test_data.loc[i,'Target']=y_predict[i]


In [ ]:
test_data

In [ ]:
test_data.to_csv(file_name, sep=',')

https://www.listendata.com/2018/01/linear-regression-in-python.html